In [38]:
import os
from urllib.request import urlretrieve
import numpy as np
import json
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [39]:
current_directory = os.getcwd()
json_file_path = os.path.join(current_directory, "api_response.json")
cv_file_path = os.path.join(current_directory, "CV RAZIG_Ilias_en.pdf")

with open(json_file_path, 'r') as j:
     data = json.loads(j.read())

In [40]:
data

{'resultats': [{'id': '183YYMR',
   'intitule': 'Data scientist  -  Ingénieur Support Data Science / Référent MLop (H/F)',
   'description': "Au sein de la direction Innovation Data et Digital (120 personnes, incubateur et accélérateur des projets pour tout le groupe Malakoff Humanis) vous serez rattaché.e au département Data Science.\nEquipe composée de 15 personnes et travaillant sur notre site Rue Laffitte.\n\n\nQuelles seront vos missions ?\n\nVotre mission consistera à développer et maintenir des cas d'usage Data Science & IA sur lesquels vous serez référent.e en autonomie ou en collaboration avec d'autres Data Scientists.\n\nVous interviendrez en transverse en intégrant des équipes produits (Data Scientists, Data Analysts, Product Managers) sur différents chantiers pour aider à la mise en œuvre des meilleures pratiques de développement et d'automatisation : vous aiderez à concevoir et à mettre en place des designs qui permettent de monter en qualité, en fiabilité et en performanc

In [46]:
# Load pdf files in the local directory

def document_loader(file, chunk_size = 700, chunk_overlap  = 50):

    loader = JSONLoader(
        file_path = file,
        jq_schema = '.resultats[]',
        text_content = False
    )

    docs_before_split = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
    )
    docs_after_split = text_splitter.split_documents(docs_before_split)
    return docs_after_split

def pdf_loader(file):
    loader = PyPDFLoader(
        file_path = file
    )

    doc = loader.load()
    return doc

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

cv = pdf_loader(cv_file_path)
docs_after_split = document_loader(json_file_path, chunk_size = 700, chunk_overlap  = 50)

In [7]:
# We load the langchain embedding model that is on hugging face
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
)

/home/onyxia/work/RAG/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

query = """Quelles sont les offres qui demandent une maîtrise du computer vision ?"""  

relevant_documents = vectorstore.similarity_search(query)
print(f'Il y a {len(relevant_documents)} documents retrouvés qui sont pertinents par rapport à la requête. Voici le premier:\n')
print(relevant_documents[0].page_content)

Il y a 4 documents retrouvés qui sont pertinents par rapport à la requête. Voici le premier:

passant par les \u00e9tudes de faisabilit\u00e9, la pr\u00e9paration des donn\u00e9es et la mod\u00e9lisation\n\u00b7        Conception et mise en place de solutions NLP, machine learning, deep learning.\n\u00b7        Conception, d\u00e9veloppement et impl\u00e9mentation d'outils ou de solutions de reporting, d'aide \u00e0 la d\u00e9cision, de visualisations de donn\u00e9es dans les domaines du customer, du digital, de la conformit\u00e9.\n\u00b7        Cartographie des processus de transformation de donn\u00e9es\n\u00b7        Etudes et choix de solutions et de plateformes big data, analytics.\n\u00b7        Veille sur l'\u00e9volution des m\u00e9thodes, des pratiques, des outils.\n


In [9]:
import transformers
import torch
from transformers import AutoTokenizer

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.54it/s]
Device set to use cpu


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Use similarity searching algorithm and return 3 most relevant documents.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
prompt = hub.pull("rlm/rag-prompt")
llm = HuggingFacePipeline(pipeline=pipeline)


# Create a RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)


/home/onyxia/work/RAG/env/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [85]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Use similarity searching algorithm and return 3 most relevant documents.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

template = """
Tu es un expert du recrutement. Un candidat a fourni son CV suivant :
{cv_text}

Voici des offres d'emploi avec leur description, leur identifiant et d'autres informations disponibles :
{context}

Utilise le CV et le contexte pour répondre à la question suivante.
{question}
"""

prompt = PromptTemplate(input_variables=["cv_text", "context", "question"], template=template)
llm = HuggingFacePipeline(pipeline=pipeline)
texte = format_docs(cv)


# Create a RAG Chain
rag_chain = (
    {
        "cv_text": RunnableLambda(lambda x: format_docs(cv)),
        "context": retriever | RunnableLambda(lambda x: format_docs(x)),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [77]:
format_docs(cv)

' RAZIG Ilias   \n \n       | razigilias@gmail.com | +33 07.83.74.23.94 \n \nEnthusiastic data scientist with a financial background, looking for an opportunity in data science. Eager to leverage my passion for machine \nlearning to drive analysis at the intersection of data science and business expertise. \n \nEDUCATION \n09/2023 – 09/2024 ▪ MS Data Science ENSAE, Institut Polytechnique de Paris \nMain courses : \n· Deep Learning : Models and Optimization \n· Reinforcement Learning \n· Machine Learning for NLP \n· Deploiement of Data Science projects \n· Machine Learning for portfolio management and trading \n· Bayesian Statistics \n· Advanced convex optimization \nParis, \nFRANCE \n09/2019 – 09/2023 ▪ Magistère Banking and Finance,  M2 Financial and Banking Techniques at PARIS University \nPanthéon-Assas. Received with high honours (15,1/20). \nMain courses : \n· Financial Markets (Derivatives, Fixed Income, Asset Valuation, Structuring) \n· Economics (Macroeconomics / Microeconomics

In [86]:
test = rag_chain.invoke("Quelle est la meilleure offre d'emploi pour ce candidat ?")

In [89]:
test

'\nTu es un expert du recrutement. Un candidat a fourni son CV suivant :\n RAZIG Ilias   \n \n       | razigilias@gmail.com | +33 07.83.74.23.94 \n \nEnthusiastic data scientist with a financial background, looking for an opportunity in data science. Eager to leverage my passion for machine \nlearning to drive analysis at the intersection of data science and business expertise. \n \nEDUCATION \n09/2023 – 09/2024 ▪ MS Data Science ENSAE, Institut Polytechnique de Paris \nMain courses : \n· Deep Learning : Models and Optimization \n· Reinforcement Learning \n· Machine Learning for NLP \n· Deploiement of Data Science projects \n· Machine Learning for portfolio management and trading \n· Bayesian Statistics \n· Advanced convex optimization \nParis, \nFRANCE \n09/2019 – 09/2023 ▪ Magistère Banking and Finance,  M2 Financial and Banking Techniques at PARIS University \nPanthéon-Assas. Received with high honours (15,1/20). \nMain courses : \n· Financial Markets (Derivatives, Fixed Income, Ass